In [1]:
!pip install -q U langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.5 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import os
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
from pinecone import Pinecone, ServerlessSpec
pinecone_object = Pinecone(api_key=PINECONE_API_KEY)


In [3]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [55]:
index_name = 'hair-rag-project-new'
pinecone_object.create_index(
                name=index_name,
                dimension=768,
                metric='cosine',
                spec=ServerlessSpec(cloud='aws',region='us-east-1')
)
index= pinecone_object.Index(index_name)

In [23]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index,embedding=embedding)

In [56]:
from langchain_core.documents import Document
document_1=Document(page_content="How do I track my order? You can track your order by logging into your account and going to the 'Order History' section.")
document_2=    Document(page_content="What is your return policy? Our return policy allows returns within 30 days of purchase.")
document_3 = Document(page_content="Can I cancel or modify my order after placing it? Yes, you can cancel or modify your order within 24 hours of placing it by contacting our customer support team.")
document_4 = Document(page_content="What should I do if I receive a damaged or incorrect item? If you receive a damaged or incorrect item, please contact our support team immediately with your order details and a photo of the item.")
document_5 = Document(page_content="Do you offer discounts or promotional codes? Yes, we regularly offer discounts and promotional codes. Sign up for our newsletter or follow us on social media to stay updated.")
document_6 = Document(page_content="How long does delivery take? Delivery typically takes 3-7 business days for domestic orders and 7-14 business days for international orders.")
documents =[document_1,document_2,document_3,document_4,document_6,document_5]

In [60]:
random_ids = [str(index) for index, _ in enumerate(documents)]

response = vector_store.add_documents(documents=documents, ids=random_ids)
print(response)

['0', '1', '2', '3', '4', '5']


6


In [71]:

results2 = vector_store.similarity_search_with_score(
"What is the procedure for returning items?")
for res,score in results2:
    if score > 0.65:
        print(f"score:{score}, page_content:{res.page_content}, metadata:{res.metadata}")
    print("IN general")
    print(f"score:{score},page_content:{res.page_content},metadata:{res.metadata}")

score:0.687536061, page_content:What is your return policy? Our return policy allows returns within 30 days of purchase., metadata:{}
IN general
score:0.687536061,page_content:What is your return policy? Our return policy allows returns within 30 days of purchase.,metadata:{}
IN general
score:0.64762485,page_content:What should I do if I receive a damaged or incorrect item? If you receive a damaged or incorrect item, please contact our support team immediately with your order details and a photo of the item.,metadata:{}
IN general
score:0.606274903,page_content:How do I track my order? You can track your order by logging into your account and going to the 'Order History' section.,metadata:{}
IN general
score:0.603907764,page_content:Can I cancel or modify my order after placing it? Yes, you can cancel or modify your order within 24 hours of placing it by contacting our customer support team.,metadata:{}


In [81]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [79]:

from langchain.prompts import PromptTemplate
def answer_to_user(query:str):
  context_with_scores =vector_store.similarity_search_with_score(query)
  filtered_context = [res.page_content for res, score in context_with_scores if score > 0.65]
  # if not filtered_context:
  #       filtered_context = ["I’m sorry, I don’t know the answer to that right now."]

  prompt_template = PromptTemplate(
            input_variables=["filtered_context", "user_query"],
            template="""
                You are a helpful and knowledgeable chatbot for an e-commerce website. Your job is to answer customer questions based on the provided context. If the answer cannot be found in the context, respond politely with: "I’m sorry, I don’t know the answer to that right now."

                ### Context:
                {filtered_context}

                ### User Question:
                {user_query}

                ### Guidelines for Responses:
                1. Answer based on the provided context.
                2. Be concise, friendly, and professional.
                3. If unsure, say: "I’m sorry, I don’t know the answer to that right now."
                4. Offer help where possible, like checking stock, suggesting products, or explaining policies.

                ### Response:
            """
        )

  formatted_prompt = prompt_template.format(filtered_context=filtered_context, user_query=query)
  final_result = llm.invoke(formatted_prompt)
  return final_result


In [83]:
final_answer=answer_to_user('what is your return policy')
print(final_answer.content)

Our return policy allows returns within 30 days of purchase.



In [85]:
final_answer=answer_to_user('my order is broken how we can register the complain ')
print(final_answer.content)

Please contact our support team immediately with your order details and a photo of the damaged item.



In [86]:
final_answer=answer_to_user('wrannty ')
print(final_answer.content)

I’m sorry, I don’t know the answer to that right now.



In [ ]:
# Get the index statistics
index_stats = index.describe_index_stats()

# Print the stats
print(index_stats)


In [ ]:
fetched_vector = index.fetch(ids=random_ids)
print(len(fetched_vector['vectors']))  # Count the number of fetched vectors
